# Stacking about

* https://adventofcode.com/2022/day/5

Today we are moving things between stacks, so naturally, this can be implemented using [stacks](https://en.wikipedia.org/wiki/Stack_(abstract_data_type)).

Most of the code went into parsing the puzzle input and being able to output it again. :-) Parsing the input involved reversing the input lines (so the bottoms of the stacks are in the first line, skipping the stack numbers), taking every 4th character starting at the 2nd column (these are all the letters on the crates, or spaces when past the top-most crate), and then [transposing the rows to columns using `zip()`](https://stackoverflow.com/questions/6473679/transpose-list-of-lists). Viola, lists of each of the input stack sequence of letters, from bottom-most to top-most crate.

In [1]:
import re
from dataclasses import dataclass
from typing import NamedTuple, Self


@dataclass
class Stacks:
    stacks: tuple[list[str]]

    def __str__(self) -> str:
        stacks = [list(stack) for stack in self.stacks]
        max_height = max(len(stack) for stack in stacks)
        cols = []
        for i, stack in enumerate(stacks, 1):
            if cols:
                cols.append(" " * (max_height + 1))
            padding = " " * (max_height - len(stack))
            cols.append(" " + "[" * len(stack) + padding)
            cols.append(str(i) + "".join(stack) + padding)
            cols.append(" " + "]" * len(stack) + padding)
        lines = ["".join(line) for line in zip(*cols)]
        return "\n".join(lines[::-1])

    @classmethod
    def from_text(cls, text: str) -> Self:
        # drop the number line, reverse, and only keep the letters columns
        lines = [line[1::4] for line in text.splitlines()[-2::-1]]
        stacks = [[letter for letter in col if letter != " "] for col in zip(*lines)]
        return cls(tuple(stacks))

    @property
    def tops(self) -> str:
        return "".join([stack[-1] for stack in self.stacks])

    def move_crates(self, source: int, target: int, count: int = 1) -> None:
        source_stack, target_stack = self.stacks[source], self.stacks[target]
        target_stack += source_stack[-count:]
        del source_stack[-count:]


_move: re.Pattern[str] = re.compile(
    r"move (?P<count>\d+) from (?P<source>\d+) to (?P<target>\d+)"
)


class CraneMove(NamedTuple):
    count: int
    source: int
    target: int

    def __str__(self) -> str:
        return f"move {self.count} from {self.source + 1} to {self.target + 1}"

    @classmethod
    def from_line(cls, line: str) -> Self:
        match = _move.search(line)
        assert match is not None
        params = {k: int(v) for k, v in match.groupdict().items()}
        params["source"] -= 1
        params["target"] -= 1
        return cls(**params)


@dataclass
class Crane9000Moves:
    moves: list[CraneMove]

    def __str__(self) -> str:
        return "\n".join([str(move) for move in self.moves])

    @classmethod
    def from_text(cls, text: str) -> Self:
        return cls([CraneMove.from_line(line) for line in text.splitlines()])

    def rearrange(self, stacks: Stacks):
        for move in self.moves:
            for _ in range(move.count):
                stacks.move_crates(move.source, move.target)


example: str = """\
    [D]    
[N] [C]    
[Z] [M] [P]
 1   2   3 

move 1 from 2 to 1
move 3 from 1 to 3
move 2 from 2 to 1
move 1 from 1 to 2
"""

test_stacks = Stacks.from_text(example.partition("\n\n")[0])
Crane9000Moves.from_text(example.partition("\n\n")[-1]).rearrange(test_stacks)
assert test_stacks.tops == "CMZ"

In [2]:
import aocd


starting_positions, _, procedure = aocd.get_data(day=5, year=2022).partition("\n\n")

stacks = Stacks.from_text(starting_positions)
crane = Crane9000Moves.from_text(procedure)
crane.rearrange(stacks)
print("Part 1:", stacks.tops)

Part 1: VWLCWGSDQ


## Part 2, more efficient movements

There were two ways to handle the new requirements to move whole substacks:

- Continuing using stacks, by popping the crates one by one to an _intermediary_ stack and then popping from there to the target stack.
- Refactor and switch to lists that I can just slice. I did the latter.

In [3]:
@dataclass
class Crane9001Moves(Crane9000Moves):
    def rearrange(self, stacks: Stacks):
        for move in self.moves:
            stacks.move_crates(move.source, move.target, move.count)


test_stacks = Stacks.from_text(example.partition("\n\n")[0])
Crane9001Moves.from_text(example.partition("\n\n")[-1]).rearrange(test_stacks)
assert test_stacks.tops == "MCD"

In [4]:
stacks = Stacks.from_text(starting_positions)
crane = Crane9001Moves.from_text(procedure)
crane.rearrange(stacks)
print("Part 2:", stacks.tops)

Part 2: TCGLQSLPW
